In [88]:
import torch_geometric
import torch
import numpy as np
import os
import sys

sys.path.append('../')

In [89]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader

dataName = 'PubMed'
dataset = Planetoid(root=f'./{dataName}', name=dataName, pre_transform=NormalizeFeatures())


In [90]:
trainData = dataset
testData = dataset

In [91]:
from engine.model.GCN import SimpleGCN

In [92]:
model = SimpleGCN(dataset[0].x.shape[1], torch.max(dataset[0].y).detach().numpy() +1)

In [93]:
model.train()

dataloader = DataLoader(trainData, batch_size= 1)

In [94]:
print(dataset[0])

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [95]:
print(model)

SimpleGCN(
  (conv1): GCNConv(500, 256)
  (conv2): GCNConv(256, 16)
  (conv3): GCNConv(16, 3)
)


In [96]:
from tqdm import tqdm
import logging

optimizer = torch.optim.AdamW(params = model.parameters(),lr = 0.001)
pbar = tqdm(range(500))
for e in pbar:
    optimizer.zero_grad()
    output = model(trainData[0].x, trainData[0].edge_index)
    loss = torch.nn.functional.nll_loss(output[dataset.train_mask], dataset.y[dataset.train_mask])
    loss.backward()
    optimizer.step()
    pbar.set_postfix({'loss' : loss.item()})
        
    

100%|██████████| 500/500 [03:39<00:00,  2.28it/s, loss=0.0133] 


In [97]:
model.eval()
pred = model(dataset.x, dataset.edge_index).argmax(dim=1)
correct = (pred[dataset.test_mask] == dataset.y[dataset.test_mask]).sum()
acc = int(correct) / int(dataset.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7680


In [98]:
torch.save(model, f'../data/checkpoint/simpleGCN_{dataName}.pt')